In [1]:
%matplotlib inline
import os, sys
import lxml
from io import StringIO
from lxml import etree
from simtk import unit
import pickle
import parmed as pm
import simtk.openmm as mm
import simtk.openmm.app as app
from simtk.openmm.app import StateDataReporter
import csv
import numpy as np
import matplotlib.pylab as plt
import toml
gaff_default = os.path.join("../data/gaff2.xml")
from openeye import oechem
import openmoltools as omtff
from openmoltools import forcefield_generators


In [2]:
settings = toml.load(open('sb2.toml', "r"))
settings['pH'] = 7.8

In [3]:
def run_dynamics_with_openMM(sim, output_filepath, name, nr_of_steps):
    # Set up the reporters to report energies and coordinates every 100 steps

    print('Writing trajectory to: ', output_filepath)
    print('as: ', name)
    sim.reporters.append(
        StateDataReporter(output_filepath + '/' + name + '.csv', 100, step=True, potentialEnergy=True,
                          kineticEnergy=True, temperature=True,
                          volume=True, density=True)
        )
    sim.reporters.append(app.DCDReporter(output_filepath + '/' + name + '.dcd', 100))

    # Run dynamics
    print('Running dynamics')
    sim.step(int(nr_of_steps))


# write out confs that are in save_confs list
# and start vacuum md
def create_hydrogen_definitions(
    xmltree
):
    """
    Generates hydrogen definitions for a small molecule residue template.

    """
    hydrogen_definitions_tree = etree.fromstring("<Residues/>")
    for residue in xmltree.xpath("Residues/Residue"):
        hydrogen_file_residue = etree.fromstring("<Residue/>")
        hydrogen_file_residue.set("name", residue.get("name"))
        # enumerate hydrogens in this list
        hydrogens = list()
        # Loop through atoms to find all hydrogens
        for bond in residue.xpath("Bond"):
            atomname1 = bond.get("atomName1")
            atomname2 = bond.get("atomName2")
            if 'H' in atomname1:
                # H is the first, parent is the second atom
                hydrogens.append(tuple([atomname1, atomname2]))
            elif 'H' in atomname2:
                # H is the second, parent is the first atom
                hydrogens.append(tuple([atomname2, atomname1]))

        # Loop through all hydrogens, and create definitions
        for name, parent in hydrogens:
            h_xml = etree.fromstring("<H/>")
            h_xml.set("name", name)
            h_xml.set("parent", parent)
            hydrogen_file_residue.append(h_xml)
        hydrogen_definitions_tree.append(hydrogen_file_residue)
    # Write output
    return hydrogen_definitions_tree



def create_bond_definitions(
    inputfile: str,
    residue_name : str = None
    ):
    """
    Generates bond definitions for a small molecule template to subsequently load 
    the bond definitions in the topology object. BE CAREFULL: The residue name 
    of the pdb file must match the residue name in the bxml file.

    Parameters
    ----------
    inputfile - a forcefield XML file defined using Gaff atom types
    """

    xmltree = etree.parse(
        inputfile, etree.XMLParser(remove_blank_text=True, remove_comments=True)
    )
    # Output tree
    bond_definitions_tree = etree.fromstring("<Residues/>")
    bonds = set()

    for residue in xmltree.xpath("Residues/Residue"):
        # Loop through all bonds
        bond_file_residue = etree.fromstring("<Residue/>")
        bond_file_residue.set("name", f"{residue_name}")
        for bond in residue.xpath("Bond"):
            atomname1 = bond.get("atomName1")
            atomname2 = bond.get("atomName2")
            if atomname1.startswith('H') or atomname2.startswith('H'):
                continue
            bonds.add(tuple([atomname1, atomname2]))
        
        for a1, a2 in bonds:
            b_xml = etree.fromstring("<Bond/>")
            b_xml.set("from", a1)
            b_xml.set("to", a2)
            bond_file_residue.append(b_xml)
        bond_definitions_tree.append(bond_file_residue)

    return bond_definitions_tree


In [6]:
forcefield = app.ForceField("amber10.xml", gaff_default, './sb2-test/sb2_t1.ffxml', "tip3p.xml")
pdb_object = app.PDBxFile('./sb2-test/sb2-water.cif')


system = forcefield.createSystem(
    pdb_object.topology,
    nonbondedMethod=app.PME,
    rigidWater=True,
)
system.addForce(mm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin))

integrator = mm.LangevinIntegrator(
                300*unit.kelvin,       # Temperature of heat bath
                1.0/unit.picoseconds,  # Friction coefficient
                2.0*unit.femtoseconds, # Time step
)


sim = app.Simulation(pdb_object.topology, system, integrator)
sim.context.setPositions(pdb_object.positions)



run_dynamics_with_openMM(sim, './sb2-test/', 'sb2_t1', 50000)

Writing trajectory to:  ./sb2-test/
as:  sb2_t1
Running dynamics


KeyboardInterrupt: 

In [ ]:
forcefield = app.ForceField("amber10.xml", gaff_default, './sb2-test/sb2_t2.ffxml', "tip3p.xml")
pdb_object = app.PDBxFile('./sb2-test/sb2-water.cif')


system = forcefield.createSystem(
    pdb_object.topology,
    nonbondedMethod=app.PME,
    rigidWater=True,
)
system.addForce(mm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin))

integrator = mm.LangevinIntegrator(
                300*unit.kelvin,       # Temperature of heat bath
                1.0/unit.picoseconds,  # Friction coefficient
                2.0*unit.femtoseconds, # Time step
)


sim = app.Simulation(pdb_object.topology, system, integrator)
sim.context.setPositions(pdb_object.positions)



run_dynamics_with_openMM(sim, './sb2-test/', 'sb2_t2', 50000)

In [ ]:
e_t1 = []
for idx, l in enumerate(open('./sb2-test/sb2_t1.csv', 'r')):
    if idx == 0:
        continue
    e_t1.append(float(l.split(',')[1]))
    
e_t2 = []
for idx, l in enumerate(open('./sb2-test/sb2_t2.csv', 'r')):
    if idx == 0:
        continue
    e_t2.append(float(l.split(',')[1]))

In [ ]:
plt.hist(e_t1)
plt.hist(e_t2)
plt.show()

In [ ]:
%cat /home/mwieder/Work/Projects/protons-dev/protons/examples/tautomers/sb2/input/sb2.hxml


In [ ]:
%cat /home/mwieder/Work/Projects/protons-dev/protons/examples/tautomers/sb2/output/sb2.ffxml
